In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

Given only the origin and the destination, there can be multiple paths to reach the destination. Most data-driven
methods ignore this by assuming a known, fixed noise model for travel times.

Previous Sol:  assuming a known, fixed noise model for travel times which in most instances is a unimodal
Gaussian distribution for travel times

Instead of Point estimation, multiple routes should be considered

In [9]:
data = pd.read_csv("./data/tt_data_for_ml.csv")[:10000]
y = data["t_travtime"]
X = data.drop(["t_travtime", "id"], axis = 1)
y = np.array(y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

,lat_d,lon_d,lat_o,lon_o,od_dist,year,t_time_start,o_type_ageb,o_type_dist,d_type_ageb,d_type_dist,taxi,wt_tottrips,ageb_eod1990,t_distr_o,t_distr_d
0,19.378813,-99.119835,19.436481,-99.160004,7.732718,1994,1600.0,0,1,1,0,0,281.0,9007032,2,13
1,19.436481,-99.160004,19.378813,-99.119835,7.732718,1994,730.0,1,0,0,1,0,257.0,9007032,13,2
2,19.378813,-99.119835,19.376709,-99.117508,1.423484,1994,805.0,0,1,1,0,0,324.0,9007032,13,13
3,19.378813,-99.119835,19.376709,-99.117508,1.423484,1994,1350.0,0,1,1,0,0,498.0,9007032,13,13
4,19.355120,-99.067688,19.378813,-99.119835,6.098191,1994,700.0,1,0,0,1,0,319.0,9007032,13,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19.432758,-99.193573,19.372831,-99.296509,12.788033,1994,800.0,1,0,0,1,0,48.0,15037022,101,16
9996,19.372831,-99.296509,19.409069,-99.260208,5.582102,1994,1400.0,0,1,1,0,0,66.0,15037022,101,101
9997,19.372831,-99.296509,19.498537,-99.225464,15.821566,1994,1700.0,0,1,1,0,0,52.0,15037022,110,101
9998,19.372831,-99.296509,19.490404,-99.137627,21.172990,1994,1900.0,0,1,1,0,0,52.0,15037022,26,101


In [11]:
X = np.array(X)

(10000, 16)

In [3]:
# print(torch.cuda)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
# device = torch.device("cuda")
print(f"Using {device} device")
# USE GPU to train the data
# CUDA 12.3 toolkit
# Now I use CPU, Install the CUDA 12.3 toolkit later

False
Using cuda device


In [4]:
# Define the mixture density network (MDN) model
class MDN(nn.Module):
    def __init__(self, input_size, hidden_size, num_components):
        super(MDN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_components)
        self.fc3 = nn.Linear(hidden_size, num_components)
        self.fc4 = nn.Linear(hidden_size, num_components)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # SiLU
        alpha = torch.softmax(self.fc2(x), dim=1)
        mu = self.fc3(x)
        sigma = torch.exp(self.fc4(x))
        return alpha, mu, sigma

In [5]:
# Convert data to PyTorch tensors

# Sigma = 5
# What are the priors for πk and σk in Eq 2?
# The priors are intended to have ”uniform” values for wk and σk because they tend
# to go to zero due to numerical issues. You can use π ∼ Dirichlet(2) and
# σ ∼ Gamma(2, 2)


X_tensor = torch.from_numpy(X).float()
T_tensor = torch.from_numpy(y).float()

# Hyperparameters
input_size = 1
hidden_size = 64

num_components = 3
num_epochs = 1000
learning_rate = 0.001

lambda_reg = 0.01

# Instantiate the MDN model
model = MDN(input_size, hidden_size, num_components).to(device)
# 1 * 64 * 64 * 64
# Instantiate an optimizer for the MDN model
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Define the log-likelihood function loss function
def log_likelihood(alpha, mu, sigma, target):
    # Gaussian Distribution
    exponent = -0.5 * ((target - mu) / sigma)**2
    coef = 1 / (sigma * torch.sqrt(2 * np.pi))

    density = coef * torch.exp(exponent)
    weighted_density = torch.sum(alpha * density, dim=1)

    log_density = torch.log(weighted_density + 1e-9)  
    # Adding epsilon for numerical stability
    
    return torch.mean(log_density)


# Training loop
for epoch in range(num_epochs):
    optimizer.zero_grad()
    alpha, mu, sigma = model(X_tensor)
    loss_likelihood = -log_likelihood(alpha, mu, sigma, T_tensor)
    loss_reg = lambda_reg * torch.sum(torch.log(alpha) + torch.log(sigma))  # Regularization term
    loss = loss_likelihood + loss_reg
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Generate samples from the model
with torch.no_grad():
    alpha, mu, sigma = model(X_tensor)
    component_idx = torch.multinomial(alpha, 1)
    samples = torch.normal(mu.gather(1, component_idx), sigma.gather(1, component_idx))
    
# Print the generated samples
print('Generated Samples:')
print(samples[:10])

TypeError: expected np.ndarray (got DataFrame)